In [41]:
import numpy as np
import tensorflow as tf

In [42]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L, preprocess_input as pp
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## Load npz file and inspect its folder structure.

In [43]:
npz = np.load('/kaggle/input/plants/public_data.npz', allow_pickle=True)
print(npz.files)

['data', 'labels']


In [44]:
data = npz['data']
labels = npz['labels']

## Convert string labels into numerical values.

In [45]:
binary_labels = np.array([0 if label == 'healthy' else 1 for label in labels])

## Remove outliers (shrek and troll) from the dataset.

In [46]:
def remove_outliers(images, labels):
    outliers = []
    for i, image in enumerate(images):
        if np.sum(data[506] - image) == 0 or np.sum(data[338] - image) == 0:
            outliers.append(i)
    
    return np.delete(images, outliers, axis=0), np.delete(labels, outliers)
    
data, binary_labels = remove_outliers(data, binary_labels)

## Normalise the data using the function provided by efficient net.

In [47]:
#normalisation
normalised = pp(data)

## Split the dataset into train, validation and test sets.

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_val_test, y_train, y_val_test = train_test_split(normalised, binary_labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

## Apply data augmentation.

In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size=32

# Create an instance of ImageDataGenerator with desired augmentations
data_gen = ImageDataGenerator(
    # rotation_range=20,       # Degree range for random rotations
    # width_shift_range=0.2,   # Range (as a fraction of total width) for horizontal shifts
    # height_shift_range=0.2,  # Range (as a fraction of total height) for vertical shifts
    # shear_range=0.1,         # Shear Intensity (Shear angle in counter-clockwise direction)
    # zoom_range=0.1,          # Range for random zoom
    horizontal_flip=True,    # Randomly flip inputs horizontally
    vertical_flip = True,
    fill_mode='nearest'      # Strategy for filling in newly created pixels
)

# Apply the data generator to the training data
train_generator = data_gen.flow(normalised, binary_labels, batch_size=batch_size)

## Compute class weights to balance the dataset.

In [50]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"Class weights: {class_weights_dict}")

Class weights: {0: 0.8035796236805874, 1: 1.3235071806500378}


In [51]:
from tensorflow.keras.models import load_model

class model:
    def __init__(self, path):
        self.model = load_model(path)
        
    def predict(self, X):
        X_processed = self._preprocess(X)
        predictions = self.model.predict(X_processed)
        predictions = self._postprocess(predictions)
        return predictions
    
    def _preprocess(self, X):
        return pp(X)
    
    def _postprocess(self, predictions):
        return (predictions > 0.5).astype(int)

## Squeeze-and-Excitation Block

Integrating the SE block right after the base model and before our custom layers allows us to refine the feature maps output by the base model. By doing so, our network can learn to focus more on the most discriminative features extracted from the input data, which can potentially lead to better performance of our final classification or regression task.


In [52]:
def squeeze_excite_block(input_tensor, ratio=16):
    init = input_tensor
    channel_axis = -1  # Assuming channels-last format
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape(se_shape)(se)

    return Multiply()([init, se])

## Load base model.
## Apply squeeze and excite.
## Add final layers.

In [53]:
def build_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Load EfficientNetV2L with pretrained weights
    base_model = EfficientNetV2L(include_top=False, weights=None, input_tensor=inputs)
    base_model.load_weights('/kaggle/input/efficientnet-imagenet/efficientnetv2-l_notop.h5')

    # Freeze the base model
    base_model.trainable = False

    # Add SE blocks and build the model
    x = base_model.output
    x = squeeze_excite_block(x)

    # Add final layers for binary classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=output)
    return model

In [56]:
# Assuming input shape of (96, 96, 3) for example
model = build_model((96, 96, 3))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Fit the model.

In [57]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set the number of epochs and batch size
epochs = 15

# Define the EarlyStopping and ModelCheckpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/kaggle/working/best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
history = model.fit(
    train_generator, 
    epochs=epochs, 
    #batch_size=batch_size, 
    #validation_data=(X_val, y_val), 
    class_weight=class_weights_dict, 
    #callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# Load the best saved model
model.load_weights('/kaggle/working/best_model.h5')

Epoch 1/15


2023-11-09 23:19:13.543107: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


157/157 [==============================] - 43s 68ms/step - loss: 0.5110 - accuracy: 0.7372
Epoch 2/15
157/157 [==============================] - 11s 68ms/step - loss: 0.4373 - accuracy: 0.7970
Epoch 3/15
157/157 [==============================] - 11s 67ms/step - loss: 0.4077 - accuracy: 0.8187
Epoch 4/15
157/157 [==============================] - 11s 67ms/step - loss: 0.3869 - accuracy: 0.8229
Epoch 5/15
157/157 [==============================] - 11s 67ms/step - loss: 0.3743 - accuracy: 0.8331
Epoch 6/15
157/157 [==============================] - 11s 67ms/step - loss: 0.3605 - accuracy: 0.8381
Epoch 7/15
157/157 [==============================] - 11s 67ms/step - loss: 0.3476 - accuracy: 0.8495
Epoch 8/15
157/157 [==============================] - 11s 68ms/step - loss: 0.3425 - accuracy: 0.8485
Epoch 9/15
157/157 [==============================] - 11s 68ms/step - loss: 0.3306 - accuracy: 0.8573
Epoch 10/15
157/157 [==============================] - 11s 67ms/step - loss: 0.3164 - accurac

## Evaluate on the whole test set.

In [20]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

24/24 [==============================] - 2s 93ms/step - loss: 0.3282 - accuracy: 0.8602
Test loss: 0.3282317519187927
Test accuracy: 0.8601863980293274


## Evaluate on different small test sets with multiple runs of this cell.

In [40]:
# Evaluate the model on the test set
X_n, X_m, y_n, y_m = train_test_split(X_test, y_test, test_size=0.17)

test_loss, test_accuracy = model.evaluate(X_m, y_m, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

4/4 [==============================] - 0s 94ms/step - loss: 0.3653 - accuracy: 0.8516
Test loss: 0.3653242886066437
Test accuracy: 0.8515625


In [61]:
model.save('/kaggle/working/eff-w-ft-fd')

In [62]:
!zip -r eff.zip /kaggle/working/eff-w-ft-fd

  adding: kaggle/working/eff-w-ft-fd/ (stored 0%)
  adding: kaggle/working/eff-w-ft-fd/variables/ (stored 0%)
  adding: kaggle/working/eff-w-ft-fd/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: kaggle/working/eff-w-ft-fd/variables/variables.index (deflated 80%)
  adding: kaggle/working/eff-w-ft-fd/fingerprint.pb (stored 0%)
  adding: kaggle/working/eff-w-ft-fd/keras_metadata.pb (deflated 96%)
  adding: kaggle/working/eff-w-ft-fd/saved_model.pb (deflated 92%)
  adding: kaggle/working/eff-w-ft-fd/assets/ (stored 0%)
